In [ ]:
import tensorflow as tf
import supporting_functions as sf
import tensorflow.keras.layers as L
from tensorflow.keras.applications import DenseNet121

IMAGE_SIZE, BATCH_SIZE = sf.get_sizes()
EPOCHS = 3

print(IMAGE_SIZE)
print(BATCH_SIZE)
print(EPOCHS)

def DenseNet():
    model  = DenseNet121(input_shape=(*IMAGE_SIZE, 3), weights='imagenet', include_top=False)
    return model

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# hardcoded cuz I'm a hack
NUM_TRAINING_IMAGES = 33126
NUM_TEST_IMAGES = 10982

TRAIN_STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
TEST_STEPS = NUM_TEST_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} test images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))

In [ ]:
# Defining the optimizer and compiling the model
with strategy.scope():
    model = tf.keras.Sequential([
        DenseNet(),
        L.GlobalAveragePooling2D(),
        #L.Dense(1024, activation = 'relu'),
        #L.Dropout(0.3) , 
        L.Dense(512, activation= 'relu'), 
        L.Dropout(0.25), 
        #L.Dense(256, activation='relu'), 
        #L.Dropout(0.2), 
        #L.Dense(128, activation='relu'), 
        #L.Dropout(0.15), 
        L.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=[tf.keras.metrics.AUC()]
    )

In [ ]:
#f = open("./model_summary.txt", "w")

#model.summary(print_fn=lambda x: f.write(x + '\n'))
#f.close()

#model.get_layer(index=0).summary()

In [ ]:
# Fitting the model
history = model.fit(
    sf.get_training_dataset(),
    steps_per_epoch=TRAIN_STEPS_PER_EPOCH,
    epochs=EPOCHS,
    use_multiprocessing=False
    #validation_data=get_validation_dataset(),
    #validation_steps=VAL_STEPS_PER_EPOCH
)

In [ ]:
#model.save('model.h5')

In [ ]:
#test_predictions = model.predict(sf.get_testing_dataset(), steps=TEST_STEPS, use_multiprocessing=True)
#print(test_predictions)

In [ ]:
#GCS_PATH = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
#TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/tfrecords/test.tfrec')

#for example in tf.python_io.tf_record_iterator("../input/siim-isic-melanoma-classification/tfrecords/train00-2071.tfrec"):
#    print(tf.train.Example.FromString(example))